In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
# Activate mypy type checking
%load_ext nb_mypy

In [ ]:
import nbimporter # Activate import functionality for notebooks

# Chess-AI Statistik

Dieses Notebook lässt verschiedene AI-Versionen gegeneinander spielen und fasst die Ergebnisse in einer CSV-Datei zusammen.

Alle Optionen für die Ausführung können über die Umgebungsvariablen gesetzt werden. Das Notebook kann auch unbeaufsichtigt über [GitHub Actions](https://github.com/piuswalter/Chess-AI/actions/workflows/statistics.yml) ausgeführt werden. Der Workflow kann dort auf jedem existierenden Branch manuell ausgeführt werden. Der gewählte Branch entscheidet über den verwendeten Code. Ein Durchlauf mit den Standardoptionen (`development`-Branch) dauert derzeit etwa 1 Stunde. Die maximal erlaubte Laufzeit mit GitHub Actions beträgt 6 Stunden.

Verfügbare Versionen:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `simple evaluation function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish Engine gesteuert wird.

## Optionen manuell anpassen

Die folgende Zelle kann dazu genutzt werden um die Optionen manuell anzupassen.

In [ ]:
# Use the following lines to customize a manual run
import os
#os.environ["players"] = 'Exercise01AI, Exercise02AI, Exercise03AI, Exercise04AI, StockfishPlayer'
#os.environ["opponents"] = 'Exercise01AI, Exercise02AI, Exercise03AI, Exercise04AI, StockfishPlayer'
#os.environ["seed"] = "42"
#os.environ["repetitions"] = "2"
#os.environ["depth"] = "2"
# Use 'del os.environ["seed"]' to unset env vars in python

In [ ]:
# Import all versions
from AIBaseClass import ChessAI
from Exercise01AI import Exercise01AI
from Exercise02AI import Exercise02AI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI
from StockfishPlayer import StockfishPlayer
from Main import run_games

available_versions = [Exercise01AI, Exercise02AI, Exercise03AI, Exercise04AI, StockfishPlayer]

Die `cleanup_game_folder`-Funktion löscht alle Dateien im `games`-Ordner. Dies verhindert, dass bereits existierende Dateien ungewollt in den Ergebnissen enthalten sind.

In [ ]:
import os

def cleanup_game_folder() -> None:
    """Deletes all files inside the games folder."""
    for filename in os.listdir("../games"):
        os.remove(f"../games/{filename}")

Die `get_players`-Funktion nimmt als Argument einen String mit Klassennamen (durch Komma getrennt)
und gibt eine Liste mit den identifizierten Chess-AI-Versionen zurück.

In [ ]:
def get_players(players_string: str) -> list[ChessAI]:
    """Creates a player list from a (comma separated) players string."""
    players_str_list = [player.strip() for player in players_string.split(",")]
    return [player for player in available_versions if player.__name__ in players_str_list]

Die `build_player_pairs`-Funktion ermittelt die angegebenen Spieler und Gegner und bildet eine Liste von Spielerpaaren (kartesisches Produkt).

In [ ]:
def build_player_pairs() -> list[ChessAI]:
    """Builds the sorted cartesian product of players and opponents."""
    # Default (all ai versions)
    all_players_str = ", ".join([player.__name__ for player in available_versions])
    # Get versions from environment
    players = get_players(os.environ.get("players", all_players_str))
    opponents = get_players(os.environ.get("opponents", all_players_str))
    # Build and return pairs
    player_pairs = {(white, black) for white in players for black in opponents}
    return sorted(player_pairs, key=lambda x: (x[0].__name__, x[1].__name__))

Die `run`-Funktion nimmt eine Liste von Spielerpaaren und führt für jedes Paar die gewünschte Anzahl an Spielen mit dem gewünschten Seed durch.

In [ ]:
def run(player_pairs: list[ChessAI]):
    """Runs one or more games with each player pair."""
    seed = int(os.environ.get("seed", "3"))
    repetitions = int(os.environ.get("repetitions", "1"))
    # Let all player pairs play
    for i, players in enumerate(player_pairs):
        print(f"Playing game {i+1} of {len(player_pairs)}")
        run_games(players[0]("White"), players[1]("Black"), repetitions, seed)

Die `collect_results`-Funktion importiert die Ergebnisse aus allen `.pgn`-Dateien im `games`-Ordner und bereitet sie als Dictionary auf.

In [ ]:
# Collect results from .pgn files
import chess.pgn

def collect_results() -> dict[str, list]:
    """Scans all .pgn files inside the games folder and collects the game results as dictionary."""
    # All results are presented from the view of the white color
    results_map_white = {"1-0": "Won", "0-1": "Lost", "1/2-1/2": "Remis", "*": "Remis/Unknown"}
    games_map: dict[str, list] = {}

    # Iterate over all .pgn files
    for filename in os.listdir("../games"):
        if not filename.endswith(".pgn"):
            continue
        with open(os.path.join("..", "games", filename)) as pgn:
            pgn_game = chess.pgn.read_game(pgn)
        if not pgn_game:
            continue

        # Collect results from white perspective
        white_view = {
            "round": pgn_game.headers["Round"],
            "opponent": pgn_game.headers["Black"],
            "result": results_map_white[pgn_game.headers["Result"]],
            "outcome": pgn_game.headers["Outcome"],
            "duration": pgn_game.headers["Duration"],
            "seed": pgn_game.headers["Seed"],
            "total cache hits": sum([int(num) for num in pgn_game.headers["Cache hits"].split(" - ")]),
            "state_changes": {node.comment: node.ply() for node in pgn_game.mainline() if node.comment},
            "total_moves": pgn_game.end().ply(),
            "filename": filename,
        }

        # Save results (uses player name as key)
        entry = games_map.get(pgn_game.headers["White"], [])
        entry.append(white_view)
        games_map[pgn_game.headers["White"]] = entry
    return games_map

Die `export_to_csv`-Funktion exportiert eine gegebene Sammlung (Dictionary) von Spielergebnissen als CSV-Datei.

In [ ]:
import csv
from datetime import datetime

def export_to_csv(games_map: dict[str, list]) -> None:
    """Exports the given results dictionary to csv."""
    filename = f'statistics_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")}.csv'
    with open(os.path.join("..", "games", filename), 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        # Write headlines
        csv_writer.writerow([
            "Player", 
            "Opponent", 
            "Result",
            "Outcome",
            "Moves", 
            "Duration", 
            "Cache hits",
            "Moves to middle game", 
            "Moves to end game", 
            "Round", 
            "Seed",
            "File",
        ])
        # Write results
        for player in sorted(games_map):
            games = sorted(games_map[player], key=lambda game: game["filename"])
            for game in games:
                csv_writer.writerow([
                    player, 
                    game["opponent"], 
                    game["result"], 
                    game["outcome"].split(".")[1],
                    game["total_moves"], 
                    game["duration"].split(".")[0], 
                    game["total cache hits"],
                    game["state_changes"].get("State.MIDDLE_GAME", "-"), 
                    game["state_changes"].get("State.END_GAME", "-"), 
                    game["round"].replace("/", " of "),
                    game["seed"],
                    game["filename"],
                ])
            # Newline between different players (white)
            csv_writer.writerow([])

Die folgende Zelle bereinigt den `games`-Ordner, erstellt die Spielerpaare, startet die Spiele, sammelt die Ergebnisse und exportiert sie als CSV-Datei.

In [ ]:
%%time
# Run and export games
cleanup_game_folder()
player_pairs = build_player_pairs()
run(player_pairs)
results = collect_results()
export_to_csv(results)